## Traditional machine learning
Feature(s): 'SMILES' column

- Copolymer sequence of A and B monomer
- Binary encoding

Target: y_bin

- Binary class

In [10]:
import sys
import os

# Append the parent directory of your package to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..')))

In [11]:
import pandas as pd
import numpy as np
import zipfile

path_to_dataset = 'train_polymers.csv'
csv_filename = 'train_polymers.csv'

# Open the file, Correct the encoding and sep if necessary
if path_to_dataset.endswith('.zip'):
    with zipfile.ZipFile(path_to_dataset, 'r') as z:
        # Open the CSV file within the ZIP file
        with z.open(csv_filename) as f:
            # Read the CSV file into a DataFrame
            df = pd.read_csv(f, sep=',', on_bad_lines='warn', index_col = 0)
else:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(path_to_dataset, sep=',', on_bad_lines='warn', index_col = 0)


print('Count of unique smiles:', df.SMILES.unique().shape[0])
print('Count of all of the smiles:', df.shape[0])


Count of unique smiles: 16000
Count of all of the smiles: 16000


In [12]:
# Define mapping
mapping = {'A': 0, 'B': 1}

# Apply mapping and convert sequences to arrays
def encode_sequence(sequence):
    return np.array([mapping[char] for char in sequence.split()])

df['Encoded'] = df['SMILES'].apply(encode_sequence)

In [13]:
from MLPipeline import MLmodel, BinTheTarget

Target = ['y_bin']
Features = ['Encoded']
Feature_types = ['numerical']
input = df[:7000]

In [14]:
model = MLmodel(modelType='RandomForestClassifier',
                    df=input,
                    target=Target,
                    features=Features,
                    feature_types=Feature_types)

# get the values (input and output) of the model
X_train, X_test, y_train, y_test = model.getValues()

model.train()
model.evaluate()

2024-09-12 17:41:52.888 | INFO     | MLPipeline:__post_init__:133 - ndim y_train: 1
2024-09-12 17:41:52.889 | INFO     | MLPipeline:__post_init__:134 - ndim x_train: 2
2024-09-12 17:41:52.890 | INFO     | MLPipeline:__post_init__:135 - shape y_train: (50,)
2024-09-12 17:41:52.890 | INFO     | MLPipeline:__post_init__:136 - shape x_train: (50, 20)


RandomForestClassifier model trained successfully.
Accuracy for RandomForestClassifier: 0.66


0.66

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.base import clone

def objective(trial, model_instance):
    """
    Objective function for Optuna to minimize.
    """
    # Define hyperparameters to tune
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_categorical('max_depth', [None, 10, 20, 30, 40]),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 15),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 6),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }

    # Clone the model to ensure a fresh instance each trial
    model_clone = clone(model_instance.model)
    model_clone.set_params(**params)
    
    # Define the score metric
    scoring = 'accuracy'

    # Perform cross-validation
    scores = cross_val_score(model_clone, model_instance.X_train, model_instance.y_train, cv=model_instance.cv, scoring=scoring)

    # Return the average score across all folds
    return scores.mean()

In [16]:
model = MLmodel(modelType='RandomForestClassifier', df=df, target=Target, 
                features=Features, hyperparameter_tuning=True,
                feature_types=Feature_types,
                optimization_method='optuna', objective=lambda trial: objective(trial, model))

model.train()
predictions = model.predict()
model.evaluate()

2024-09-12 17:41:53.025 | INFO     | MLPipeline:__post_init__:133 - ndim y_train: 1
2024-09-12 17:41:53.026 | INFO     | MLPipeline:__post_init__:134 - ndim x_train: 2
2024-09-12 17:41:53.026 | INFO     | MLPipeline:__post_init__:135 - shape y_train: (50,)
2024-09-12 17:41:53.027 | INFO     | MLPipeline:__post_init__:136 - shape x_train: (50, 20)
[I 2024-09-12 17:41:53,028] A new study created in memory with name: no-name-7a3dc1a8-35b9-4e5c-aef1-7b8a39019015
[I 2024-09-12 17:41:54,114] Trial 0 finished with value: 0.82 and parameters: {'n_estimators': 217, 'max_depth': None, 'min_samples_split': 7, 'min_samples_leaf': 5, 'max_features': 'log2', 'bootstrap': True}. Best is trial 0 with value: 0.82.
[I 2024-09-12 17:41:54,565] Trial 1 finished with value: 0.8 and parameters: {'n_estimators': 126, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 5, 'max_features': 'log2', 'bootstrap': False}. Best is trial 0 with value: 0.82.
[I 2024-09-12 17:41:54,905] Trial 2 finished with v

Best RandomForestClassifier model trained successfully with hyperparameter tuning using Optuna.
Best hyperparameters: {'n_estimators': 66, 'max_depth': 30, 'min_samples_split': 8, 'min_samples_leaf': 3, 'max_features': 'sqrt', 'bootstrap': False}
RandomForestClassifier model trained successfully.
Accuracy for RandomForestClassifier: 0.76


0.76